In [2]:
import numpy as np
import os
from tensorflow import keras
from tqdm import tqdm
import cv2 , pickle

In [6]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount('/content/drive/')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
wd = './drive/My Drive/DD2412_Project/'
local_download_path = os.path.expanduser(f'{wd}ILSVRC2012_img_val')
local_download_path

'./drive/My Drive/DD2412_Project/ILSVRC2012_img_val'

In [8]:
from model import load_model
from image_utils import imagenet_utils
from explanation_models import gradcam,gradcam_robust,gradcam_plusplus,guided_backprop


# **`VGG 16`**

In [9]:
folder=os.path.join(wd,'ILSVRC2012_img_val/')
img_names = os.listdir(folder)
n = len(img_names)

model_name = 'VGG16'
test=load_model(model_name)
model=test.model
input_dim=test.input_dim
layer_name='block5_conv3'



553467904/553467096 [==============================] - 4s 0us/step


Grad-CAM

In [10]:

g= gradcam(model,layer_name,input_dim)
heatmap_dict={}

for image_name in tqdm(img_names):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input = keras.applications.vgg16.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap_list.append([index,heatmap])
    heatmap_dict[image_name]=heatmap_list

100%|██████████| 1000/1000 [07:18<00:00,  2.28it/s]


In [11]:
pickle.dump(heatmap_dict,open(os.path.join(wd,f'gradcam_heatmaps_{model_name}_{n}.p'),'wb'))

Grad-CAM++

In [12]:

g= gradcam_plusplus(model,layer_name,input_dim)
heatmap_dict={}

for image_name in tqdm(img_names):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input = keras.applications.vgg16.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap_list.append([index,heatmap])
    heatmap_dict[image_name]=heatmap_list

100%|██████████| 1000/1000 [11:17<00:00,  1.48it/s]


In [13]:
pickle.dump(heatmap_dict,open(os.path.join(wd,f'gradcam_plus_plus_heatmaps_{model_name}_{n}.p'),'wb'))

# RESNET

In [18]:
model_name = 'resnet'
test=load_model(model_name)
model=test.model
input_dim=test.input_dim
layer_name='conv5_block3_3_conv'


Grad-CAM

In [19]:

g= gradcam(model,layer_name,input_dim)
heatmap_dict={}

for image_name in tqdm(img_names):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input = keras.applications.resnet50.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap_list.append([index,heatmap])
    heatmap_dict[image_name]=heatmap_list

100%|██████████| 1000/1000 [07:25<00:00,  2.25it/s]


In [21]:
pickle.dump(heatmap_dict,open(os.path.join(wd,f'gradcam_heatmaps_{model_name}_{n}.p'),'wb'))

Grad-CAM++

In [22]:

g= gradcam_plusplus(model,layer_name,input_dim)
heatmap_dict={}

for image_name in tqdm(img_names):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input = keras.applications.resnet50.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap_list.append([index,heatmap])
    heatmap_dict[image_name]=heatmap_list

100%|██████████| 1000/1000 [12:59<00:00,  1.28it/s]


In [23]:
pickle.dump(heatmap_dict,open(os.path.join(wd,f'gradcam_plus_plus_heatmaps_{model_name}_{n}.p'),'wb'))

# InceptionV3

In [33]:
model_name = 'inceptionv3'
test=load_model(model_name)
model=test.model
input_dim=test.input_dim
layer_name='conv2d_93'

In [34]:

g= gradcam(model,layer_name,input_dim)
heatmap_dict={}

for image_name in tqdm(img_names):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input = keras.applications.inception_v3.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap_list.append([index,heatmap])
    heatmap_dict[image_name]=heatmap_list

ValueError: ignored

In [35]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 149, 149, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 149, 149, 32) 96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 149, 149, 32) 0           batch_normalization_282[0][0]    
_______________________________________________________________________________________